# import libraries

In [1]:
#standard libraries
import pandas as pd
import numpy as np

# statistic libraries
from sklearn import metrics
from sklearn.cluster import OPTICS
from sklearn.metrics import davies_bouldin_score
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import pairwise_distances

# load the complete dataset

In [2]:
df = pd.read_json('/Users/majadallacqua/Desktop/università/II_sem/CSS/urban analysis/data/trento_poi_dataset.json')
df.head()

,id,lat,lon,name,class
0,265590160,46.047611,11.126020,Volksbank,economic
1,269193707,46.074491,11.124553,Cassa di Trento,economic
2,269193731,46.064718,11.123213,Cassa di Trento,economic
3,288910331,46.065631,11.154994,Cassa di Trento,economic
4,292015813,46.076473,11.141931,Cassa di Trento,economic


# set the environment

In [3]:
#kms_per_radian = 6371.0088

# 12 is the number of districts in the city of Trento
min_neigh = 12 - 5 # value for the city of Trento
max_neigh = 12 + 5

# subset used for each analysis
df_eco = df[df['class']=='economic']
df_edu = df[df['class']=='education']
df_hea = df[df['class']=='health']
df_cat = df[df['class']=='catering']
df_shop = df[df['class']=='shopping']
df_tou = df[df['class']=='tourism']

# optics algorithm

In [4]:
# pre-computation of the harvesine distance since it is not available in the optics implementation
def haversine_distance(lat1, lon1, lat2, lon2):

    # Calcolo delle differenze di latitudine e longitudine
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    # Calcolo della distanza utilizzando la formula dell'Haversine
    a = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    r = 6371  # Raggio medio della Terra in chilometri
    distance = c * r
    
    return distance

In [5]:
# let's the value we want to try
min_sample_params = [2, 3, 4, 5, 6, 7, 8, 9] # minimum number of elements nearby a point to be considered core point
min_cluster_size_params = [2, 3, 4, 5, 6, 7, 8, 9] # min number of elements we want in each cluster

## economic dataset

In [10]:
df_eco_rad = np.array(np.radians(df_eco[['lat','lon']])) # Conversione delle coordinate in radianti

# Creazione di una matrice di distanze
n_points = len(df_eco_rad)

distance_matrix = np.zeros((n_points, n_points))
for i in range(n_points):
    for j in range(i, n_points):
        distance_matrix[i, j] = haversine_distance(df_eco_rad[i][0], df_eco_rad[i][1], df_eco_rad[j][0], df_eco_rad[j][1])
        distance_matrix[j, i] = distance_matrix[i, j]

# now we can compute the optic algorithm
scores_eco = []

for i in min_sample_params:
    
    for j in min_cluster_size_params:
        
        # general parameters for the DBSCAN algorithm
        optics = OPTICS(min_samples=i, 
                        xi=0.05, 
                        min_cluster_size=j, 
                        metric='precomputed')
        
        # fit to economic dataset
        optics_eco = optics.fit(distance_matrix)
        clusters_eco = optics_eco.labels_
        num_clusters_eco = len(set(clusters_eco)) - 1 # to esclude the noise cluster
        
        if num_clusters_eco in range(min_neigh, max_neigh + 1):
            sil_score = metrics.silhouette_score(df_eco_rad, clusters_eco)
            davies_score = davies_bouldin_score(df_eco_rad, clusters_eco)

            score_valid = (i, j, num_clusters_eco, sil_score, davies_score)
            scores_eco.append(score_valid)
        
scores_eco.sort(key=lambda tuple: tuple[2], reverse=True)     

# let's print the results to compare them:
print(f"the optics found {len(scores_eco)} valid results") # use this to check how many items have been saved    
for s in scores_eco:
    print(f"the result is:{s}")

the optics found 7 valid results
the result is:(2, 2, 15, 0.33823179856888613, 3.857171705632589)
the result is:(2, 3, 10, 0.1291152788240539, 3.6313272749807126)
the result is:(3, 2, 10, 0.17362172568474177, 4.303968956970478)
the result is:(3, 3, 9, 0.13571308532174886, 4.931786481562657)
the result is:(2, 4, 7, -0.02293910824390997, 5.802621109374607)
the result is:(4, 2, 7, -0.03507645966274629, 7.389341477013563)
the result is:(4, 3, 7, -0.03507645966274629, 7.389341477013563)


## education dataset

In [11]:
df_edu_rad = np.array(np.radians(df_edu[['lat','lon']])) # Conversione delle coordinate in radianti

# Creazione di una matrice di distanze
n_points = len(df_edu_rad)

distance_matrix = np.zeros((n_points, n_points))
for i in range(n_points):
    for j in range(i, n_points):
        distance_matrix[i, j] = haversine_distance(df_edu_rad[i][0], df_edu_rad[i][1], df_edu_rad[j][0], df_edu_rad[j][1])
        distance_matrix[j, i] = distance_matrix[i, j]

# now we can compute the optic algorithm
scores_edu = []

for i in min_sample_params:
    
    for j in min_cluster_size_params:
        
        # general parameters for the DBSCAN algorithm
        optics = OPTICS(min_samples=i, 
                        xi=0.05, 
                        min_cluster_size=j, 
                        metric='precomputed')
        
        # fit to edunomic dataset
        optics_edu = optics.fit(distance_matrix)
        clusters_edu = optics_edu.labels_
        num_clusters_edu = len(set(clusters_edu)) - 1 # to esclude the noise cluster
        
        if num_clusters_edu in range(min_neigh, max_neigh + 1):
            sil_score = metrics.silhouette_score(df_edu_rad, clusters_edu)
            davies_score = davies_bouldin_score(df_edu_rad, clusters_edu)

            score_valid = (i, j, num_clusters_edu, sil_score, davies_score)
            scores_edu.append(score_valid)
        
scores_edu.sort(key=lambda tuple: tuple[2], reverse=True)     

# let's print the results to compare them:
print(f"the optics found {len(scores_edu)} valid results") # use this to check how many items have been saved    
for s in scores_edu:
    print(f"the result is:{s}")

the optics found 5 valid results
the result is:(2, 2, 14, 0.357300198824369, 1.2244185939615952)
the result is:(2, 3, 10, 0.2238545846561928, 4.314385546032243)
the result is:(3, 2, 10, 0.18993955767520576, 2.948702235071174)
the result is:(3, 3, 10, 0.18993955767520576, 2.948702235071174)
the result is:(2, 4, 7, 0.08312932079101602, 1.7758504239152286)


## health dataset

In [12]:
df_hea_rad = np.array(np.radians(df_hea[['lat','lon']])) # Conversione delle coordinate in radianti

# Creazione di una matrice di distanze
n_points = len(df_hea_rad)

distance_matrix = np.zeros((n_points, n_points))
for i in range(n_points):
    for j in range(i, n_points):
        distance_matrix[i, j] = haversine_distance(df_hea_rad[i][0], df_hea_rad[i][1], df_hea_rad[j][0], df_hea_rad[j][1])
        distance_matrix[j, i] = distance_matrix[i, j]

# now we can compute the optic algorithm
scores_hea = []

for i in min_sample_params:
    
    for j in min_cluster_size_params:
        
        # general parameters for the DBSCAN algorithm
        optics = OPTICS(min_samples=i, 
                        xi=0.05, 
                        min_cluster_size=j, 
                        metric='precomputed')
        
        # fit to heanomic dataset
        optics_hea = optics.fit(distance_matrix)
        clusters_hea = optics_hea.labels_
        num_clusters_hea = len(set(clusters_hea)) - 1 # to esclude the noise cluster
        
        if num_clusters_hea in range(min_neigh, max_neigh + 1):
            sil_score = metrics.silhouette_score(df_hea_rad, clusters_hea)
            davies_score = davies_bouldin_score(df_hea_rad, clusters_hea)

            score_valid = (i, j, num_clusters_hea, sil_score, davies_score)
            scores_hea.append(score_valid)
        
scores_hea.sort(key=lambda tuple: tuple[2], reverse=True)     

# let's print the results to compare them:
print(f"the optics found {len(scores_hea)} valid results") # use this to check how many items have been saved    
for s in scores_hea:
    print(f"the result is:{s}")

the optics found 1 valid results
the result is:(2, 2, 7, -0.041298909616827396, 6.198425427325789)


## shopping dataset

In [13]:
df_shop_rad = np.array(np.radians(df_shop[['lat','lon']])) # Conversione delle coordinate in radianti

# Creazione di una matrice di distanze
n_points = len(df_shop_rad)

distance_matrix = np.zeros((n_points, n_points))
for i in range(n_points):
    for j in range(i, n_points):
        distance_matrix[i, j] = haversine_distance(df_shop_rad[i][0], df_shop_rad[i][1], df_shop_rad[j][0], df_shop_rad[j][1])
        distance_matrix[j, i] = distance_matrix[i, j]

# now we can compute the optic algorithm
scores_shop = []

for i in min_sample_params:
    
    for j in min_cluster_size_params:
        
        # general parameters for the DBSCAN algorithm
        optics = OPTICS(min_samples=i, 
                        xi=0.05, 
                        min_cluster_size=j, 
                        metric='precomputed')
        
        # fit to shopnomic dataset
        optics_shop = optics.fit(distance_matrix)
        clusters_shop = optics_shop.labels_
        num_clusters_shop = len(set(clusters_shop)) - 1 # to esclude the noise cluster
        
        if num_clusters_shop in range(min_neigh, max_neigh + 1):
            sil_score = metrics.silhouette_score(df_shop_rad, clusters_shop)
            davies_score = davies_bouldin_score(df_shop_rad, clusters_shop)

            score_valid = (i, j, num_clusters_shop, sil_score, davies_score)
            scores_shop.append(score_valid)
        
scores_shop.sort(key=lambda tuple: tuple[2], reverse=True)     

# let's print the results to compare them:
print(f"the optics found {len(scores_shop)} valid results") # use this to check how many items have been saved    
for s in scores_shop:
    print(f"the result is:{s}")

the optics found 4 valid results
the result is:(2, 2, 14, 0.3055166662141259, 1.9975609116247335)
the result is:(2, 3, 7, 0.12362215925071614, 2.0888125648532947)
the result is:(3, 2, 7, 0.16775706350693606, 5.78588845870898)
the result is:(3, 3, 7, 0.16775706350693606, 5.78588845870898)


## tourism dataset

In [14]:
df_tou_rad = np.array(np.radians(df_tou[['lat','lon']])) # Conversione delle coordinate in radianti

# Creazione di una matrice di distanze
n_points = len(df_tou_rad)

distance_matrix = np.zeros((n_points, n_points))
for i in range(n_points):
    for j in range(i, n_points):
        distance_matrix[i, j] = haversine_distance(df_tou_rad[i][0], df_tou_rad[i][1], df_tou_rad[j][0], df_tou_rad[j][1])
        distance_matrix[j, i] = distance_matrix[i, j]

# now we can compute the optic algorithm
scores_tou = []

for i in min_sample_params:
    
    for j in min_cluster_size_params:
        
        # general parameters for the DBSCAN algorithm
        optics = OPTICS(min_samples=i, 
                        xi=0.05, 
                        min_cluster_size=j, 
                        metric='precomputed')
        
        # fit to tounomic dataset
        optics_tou = optics.fit(distance_matrix)
        clusters_tou = optics_tou.labels_
        num_clusters_tou = len(set(clusters_tou)) - 1 # to esclude the noise cluster
        
        if num_clusters_tou in range(min_neigh, max_neigh + 1):
            sil_score = metrics.silhouette_score(df_tou_rad, clusters_tou)
            davies_score = davies_bouldin_score(df_tou_rad, clusters_tou)

            score_valid = (i, j, num_clusters_tou, sil_score, davies_score)
            scores_tou.append(score_valid)
        
scores_tou.sort(key=lambda tuple: tuple[2], reverse=True)     

# let's print the results to compare them:
print(f"the optics found {len(scores_tou)} valid results") # use this to check how many items have been saved    
for s in scores_tou:
    print(f"the result is:{s}")

the optics found 9 valid results
the result is:(2, 2, 16, 0.26664362834648003, 1.5662206762782789)
the result is:(2, 3, 9, 0.0895406212794005, 4.606519264497488)
the result is:(3, 2, 9, 0.17881723745864564, 1.5670347731316123)
the result is:(3, 3, 9, 0.17881723745864564, 1.5670347731316123)
the result is:(2, 4, 8, 0.15373673982717517, 1.7057287275232476)
the result is:(3, 4, 7, 0.10380022186309074, 1.5349515001976153)
the result is:(4, 2, 7, 0.28897795388755937, 1.647947327059121)
the result is:(4, 3, 7, 0.28897795388755937, 1.647947327059121)
the result is:(4, 4, 7, 0.28897795388755937, 1.647947327059121)


## catering  dataset

In [15]:
df_cat_rad = np.array(np.radians(df_cat[['lat','lon']])) # Conversione delle coordinate in radianti

# Creazione di una matrice di distanze
n_points = len(df_cat_rad)

distance_matrix = np.zeros((n_points, n_points))
for i in range(n_points):
    for j in range(i, n_points):
        distance_matrix[i, j] = haversine_distance(df_cat_rad[i][0], df_cat_rad[i][1], df_cat_rad[j][0], df_cat_rad[j][1])
        distance_matrix[j, i] = distance_matrix[i, j]

# now we can compute the optic algorithm
scores_cat = []

for i in min_sample_params:
    
    for j in min_cluster_size_params:
        
        # general parameters for the DBSCAN algorithm
        optics = OPTICS(min_samples=i, 
                        xi=0.05, 
                        min_cluster_size=j, 
                        metric='precomputed')
        
        # fit to catnomic dataset
        optics_cat = optics.fit(distance_matrix)
        clusters_cat = optics_cat.labels_
        num_clusters_cat = len(set(clusters_cat)) - 1 # to esclude the noise cluster
        
        if num_clusters_cat in range(min_neigh, max_neigh + 1):
            sil_score = metrics.silhouette_score(df_cat_rad, clusters_cat)
            davies_score = davies_bouldin_score(df_cat_rad, clusters_cat)

            score_valid = (i, j, num_clusters_cat, sil_score, davies_score)
            scores_cat.append(score_valid)
        
scores_cat.sort(key=lambda tuple: tuple[2], reverse=True)     

# let's print the results to compare them:
print(f"the optics found {len(scores_cat)} valid results") # use this to check how many items have been saved    
for s in scores_cat:
    print(f"the result is:{s}")

the optics found 31 valid results
the result is:(3, 8, 17, -0.13056528824436137, 4.231267681881338)
the result is:(3, 9, 17, -0.11506471087226439, 4.524717848338473)
the result is:(4, 7, 17, -0.1426207188973168, 5.071021404073162)
the result is:(5, 7, 17, -0.12324357095392269, 3.4919799305153414)
the result is:(6, 7, 17, -0.12718789695780078, 2.112624765305898)
the result is:(7, 8, 16, -0.026803207946356124, 3.6179762091018066)
the result is:(8, 2, 16, -0.025402189688466272, 2.63414647611268)
the result is:(8, 3, 16, -0.025402189688466272, 2.63414647611268)
the result is:(8, 4, 16, -0.025402189688466272, 2.63414647611268)
the result is:(8, 5, 16, -0.025402189688466272, 2.63414647611268)
the result is:(8, 6, 16, -0.025402189688466272, 2.63414647611268)
the result is:(8, 7, 16, 0.04452750194589625, 3.139181118672527)
the result is:(8, 8, 16, 0.04452750194589625, 3.139181118672527)
the result is:(2, 8, 14, -0.24391510972448846, 2.9576185924650487)
the result is:(2, 9, 14, -0.2114545468410